In [2]:
import os
import warnings
from dotenv import load_dotenv
from typing import List
from haystack import Pipeline, component
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from groq import Groq

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
load_dotenv()


True

## Custom Groq Generator


In [ ]:
@component
class GroqChatGenerator:
    def __init__(self, model: str = "llama-3.3-70b-versatile", api_key: str = None): # type: ignore
        """
        Initializes the GroqChatGenerator for chat memory.
        Updated to use current, active model.
        
        Alternative models:
        - "llama-3.1-8b-instant" (faster, smaller model)
        - "gemma2-9b-it" (Google's model)
        """
        self.client = Groq(api_key=api_key or os.environ.get("GROQ_API_KEY"))
        self.model = model
    
    @component.output_types(replies=List[ChatMessage])
    def run(self, messages: List[ChatMessage]):
        if not messages:
            raise ValueError("The 'messages' list received by GroqChatGenerator is empty.")
        
        groq_messages = []
        for msg in messages:
            content = ""
            if hasattr(msg, 'content'):
                content = msg.content
            elif hasattr(msg, 'text'):
                content = msg.text
            
            if content and hasattr(msg, 'role'):
                role = msg.role.value if hasattr(msg.role, 'value') else str(msg.role)
                groq_messages.append({"role": role.lower(), "content": content})
        
        if not groq_messages:
            raise ValueError(
                "The 'groq_messages' list is empty after conversion. "
                "The ChatMessage objects seem to be malformed (missing .role or .content/.text)."
            )
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=groq_messages,
            temperature=0.7,
            max_tokens=500
        )
        
        return {
            "replies": [
                ChatMessage.from_assistant(response.choices[0].message.content)
            ]
        }

In [12]:
print("Setting up chat message store...")
chat_message_store = InMemoryChatMessageStore()


Setting up chat message store...


## Chat Memory Components


In [6]:
chat_message_writer = ChatMessageWriter(chat_message_store)

In [7]:
chat_message_retriever = ChatMessageRetriever(chat_message_store)

In [ ]:
class ChatHistoryPipeline:
    def __init__(self, chat_message_store):
        self.chat_message_store = chat_message_store
        self.pipeline = Pipeline()
        self.pipeline.add_component("memory_retriever", ChatMessageRetriever(chat_message_store))
        self.pipeline.add_component("prompt_builder", PromptBuilder(
            template="""
            Previous Conversations history:
            {% for memory in memories %}
                {{memory.text}}
            {% endfor %}
            """,
            required_variables=["memories"]
        ))
        self.pipeline.connect("memory_retriever", "prompt_builder.memories")

    def run(self):
        res = self.pipeline.run(
            data={},
            include_outputs_from=["prompt_builder"] # type: ignore
        )
        return res["prompt_builder"]["prompt"]


### Paraphraser Pipeline (Uses Memory)


In [19]:
class ParaphraserPipeline:
    def __init__(self, chat_message_store):
        self.chat_message_store = chat_message_store
        self.memory_retriever = ChatMessageRetriever(chat_message_store)
    
    def run(self, query):
        # Get memory first
        memory_result = self.memory_retriever.run()
        memories = memory_result.get('messages', [])
        
        # Build the prompt with memories
        history_text = ""
        for memory in memories:
            content = memory.content if hasattr(memory, 'content') else memory.text
            role = memory.role.value if hasattr(memory.role, 'value') else str(memory.role)
            history_text += f"{role}: {content}\n"
        
        messages = [
            ChatMessage.from_system(
                "You are a helpful assistant that paraphrases user queries based on previous conversations."
            ),
            ChatMessage.from_user(
                f"""
                Please paraphrase the following query based on the conversation history provided below. 
                If the conversation history is empty, please return the query as is.
                If there is context from previous conversations, use that to make the query more specific.
                
                History:
                {history_text}
                
                Query: {query}
                
                Paraphrased Query:
                """
            )
        ]
        
        # Use generator directly
        generator = GroqChatGenerator()
        result = generator.run(messages)
        
        response_msg = result["replies"][0]
        return response_msg.content if hasattr(response_msg, 'content') else response_msg.text

In [20]:
print("Initializing chat memory pipelines...")
chat_history_pipeline = ChatHistoryPipeline(chat_message_store)
paraphraser_pipeline = ParaphraserPipeline(chat_message_store)

Initializing chat memory pipelines...


## Test Chat Memory Functionality


### Test 1: Empty Memory


In [24]:
print("\n=== TEST 1: EMPTY MEMORY ===")
print("Current message count:", len(chat_message_store.messages))

history = chat_history_pipeline.run()
print("Chat history (should be empty):")
print(repr(history))

query1 = "I need a smartphone"
paraphrased1 = paraphraser_pipeline.run(query1)
print(f"\nOriginal query: '{query1}'")
print(f"Paraphrased (no context): '{paraphrased1}'")


=== TEST 1: EMPTY MEMORY ===
Current message count: 0
Chat history (should be empty):
'\n            Previous Conversations history:\n            \n            '

Original query: 'I need a smartphone'
Paraphrased (no context): 'Since the conversation history is not empty, I will paraphrase the query to make it more specific. 

Paraphrased Query: I'm looking to purchase a new smartphone, can you help me find one that suits my needs?'


### Test 2: Add Messages to Memory


In [25]:
print("\n=== TEST 2: BUILDING CONVERSATION MEMORY ===")
conversation = [
    ChatMessage.from_user("Hi, I'm looking for a smartphone"),
    ChatMessage.from_assistant("Hello! I'd be happy to help you find a smartphone. What's your budget and what features are most important to you?"),
    ChatMessage.from_user("My budget is around 15 million rupiah"),
    ChatMessage.from_assistant("Great! With a 15 million rupiah budget, you have excellent options. Are you looking for any specific features like camera quality, gaming performance, or battery life?"),
    ChatMessage.from_user("I want good camera for photography"),
    ChatMessage.from_assistant("Perfect! For photography at your budget, I'd recommend looking at smartphones with advanced camera systems. Let me find some options for you.")
]

print("Adding conversation messages to memory...")
for msg in conversation:
    chat_message_writer.run([msg])

print(f"Messages in store: {len(chat_message_store.messages)}")

history = chat_history_pipeline.run()
print("\nChat history:")
print(history[:200] + "..." if len(history) > 200 else history)


=== TEST 2: BUILDING CONVERSATION MEMORY ===
Adding conversation messages to memory...
Messages in store: 0

Chat history:

            Previous Conversations history:
            
            


### Test 3: Context-Aware Paraphrasing


In [26]:
print("\n=== TEST 3: CONTEXT-AWARE PARAPHRASING ===")

test_queries = [
    "Show me some options",
    "What about cheaper alternatives?",
    "Any with better cameras?",
    "I changed my mind, show me laptops"
]

for query in test_queries:
    paraphrased = paraphraser_pipeline.run(query)
    print(f"\nOriginal: '{query}'")
    print(f"Paraphrased: '{paraphrased}'")


=== TEST 3: CONTEXT-AWARE PARAPHRASING ===

Original: 'Show me some options'
Paraphrased: 'Since the conversation history is empty except for the query "Show me some options", I will return the query as is, but I can make a slight modification to make it more general. 

Paraphrased Query: Show me some options'

Original: 'What about cheaper alternatives?'
Paraphrased: 'Since the conversation history is empty, I will return the query as is. 

Paraphrased Query: What about cheaper alternatives?'

Original: 'Any with better cameras?'
Paraphrased: 'Since the conversation history is empty except for the query "Any with better cameras?", I will return the query as is, but I can make an attempt to make it slightly more specific. 

Paraphrased Query: Are there any devices or models with improved or higher-quality camera capabilities?'

Original: 'I changed my mind, show me laptops'
Paraphrased: 'Since the conversation history is not empty, I can use the context to make the query more specific

### Test 4: Add More Context


In [27]:
print("\n=== TEST 4: EXPANDING CONVERSATION CONTEXT ===")

additional_messages = [
    ChatMessage.from_user("I like the Samsung option, tell me more about it"),
    ChatMessage.from_assistant("The Samsung Galaxy S24 is an excellent choice for photography! It features a triple camera system with advanced AI features. Would you like to know about warranty or delivery options?"),
    ChatMessage.from_user("What about delivery?")
]

for msg in additional_messages:
    chat_message_writer.run([msg])

print(f"Total messages in store: {len(chat_message_store.messages)}")

final_queries = [
    "How much does it cost?",
    "Is it available?",
    "What's the return policy?"
]

for query in final_queries:
    paraphrased = paraphraser_pipeline.run(query)
    print(f"\nOriginal: '{query}'")
    print(f"Paraphrased: '{paraphrased}'")


=== TEST 4: EXPANDING CONVERSATION CONTEXT ===
Total messages in store: 0

Original: 'How much does it cost?'
Paraphrased: 'Since the conversation history is empty, the paraphrased query is: How much does it cost?'

Original: 'Is it available?'
Paraphrased: 'Since the conversation history is empty, I will return the query as is. 

Paraphrased Query: Is it available?'

Original: 'What's the return policy?'
Paraphrased: 'Since the conversation history is empty except for the query itself, the paraphrased query would be the same as the original query. 

Paraphrased Query: What's the return policy?'


### Test 5: Memory Retrieval


In [ ]:
print("\n=== TEST 5: DIRECT MEMORY RETRIEVAL ===")

retrieval_result = chat_message_retriever.run()
print(f"Retrieved {len(retrieval_result['messages'])} messages:")

for i, msg in enumerate(retrieval_result['messages'][:5], 1):
    role = msg.role.value if hasattr(msg.role, 'value') else str(msg.role)
    content = msg.content if hasattr(msg, 'content') else msg.text
    print(f"{i}. {role}: {content[:50]}...") # type: ignore


=== TEST 5: DIRECT MEMORY RETRIEVAL ===
Retrieved 9 messages:
1. user: Hi, I'm looking for a smartphone...
2. assistant: Hello! I'd be happy to help you find a smartphone....
3. user: My budget is around 15 million rupiah...
4. assistant: Great! With a 15 million rupiah budget, you have e...
5. user: I want good camera for photography...


### Test 6: Clear Memory


In [29]:
print("\n=== TEST 6: MEMORY MANAGEMENT ===")

print(f"Messages before clear: {len(chat_message_store.messages)}")

new_chat_store = InMemoryChatMessageStore()
new_paraphraser = ParaphraserPipeline(new_chat_store)

test_query = "Show me smartphones"
new_paraphrased = new_paraphraser.run(test_query)
print(f"\nWith cleared memory:")
print(f"Original: '{test_query}'")
print(f"Paraphrased: '{new_paraphrased}'")

print(f"Messages in new store: {len(new_chat_store.messages)}")

print("\nChat memory testing completed!")


=== TEST 6: MEMORY MANAGEMENT ===
Messages before clear: 0

With cleared memory:
Original: 'Show me smartphones'
Paraphrased: 'Since the conversation history is not empty, I can use the context to make the query more specific. 

The original query is "Show me smartphones". Given that this is the starting point of the conversation, the paraphrased query can be made more specific by adding some general parameters that are often considered when looking for smartphones.

Paraphrased Query: Show me available smartphones with their prices and key features.'
Messages in new store: 0

Chat memory testing completed!
